In [1]:
import numpy as np
import pandas as pd


In [6]:
import os

In [82]:
class Decomposed_matrices:
    '''
    Read three matrices stored in csv.gz files and compute
    square cosine score and contribution score.
    
    Here are some explanation (see more details for the paper).
    
    Let's write our decomposition as X = UDV' 
    where X is input feature matrix, 
    D is diagonal singular value matrix, 
    U is left singular vector matrix (on assay space), 
    V is right singular vector matrix (on genomic bin space), 
    and `'` denotes the transposition of the matrix.
    
    Genomic bin squared cosine score is defined as L2-normalized 
    version of the matrix product (VD) so that any given slice 
    for a given genomic bin has Euclidian norm of 1. 
    
    The interpretation of the score is it represents the relative 
    importance of the component given a genomic bin.    
    '''
    def read_decomposed_matrix_file(self, filename, compression=None):
        '''
        This function reads the csv.gz file
        '''
        if((compression is None) and (len(filename) > 3) and (filename[-3:] == '.gz')):
            compression='gzip'
        df = pd.read_csv(
            os.path.join(data_dir, filename),
            compression=compression
        )
        mat = df.iloc[:, 1:].as_matrix()
        idx = df.iloc[:, 0].as_matrix()
        return mat, idx    

    def __init__(self, d_file, u_file, v_file):
        '''
        Given the file names of the three csv.gz filess, 
        load them on memory and compute the scores.
        '''
        # read three matrices D, U, V
        d_mat_temp, d_idx = self.read_decomposed_matrix_file(d_file)
        d_vec = d_mat_temp[:, 0]
        u_mat, u_idx = self.read_decomposed_matrix_file(u_file)
        v_mat, v_idx = self.read_decomposed_matrix_file(v_file)
        
        # Compute matrix produces, UD and VD
        u_dot_d = np.dot(u_mat, np.diag(d_vec))
        v_dot_d = np.dot(v_mat, np.diag(d_vec))
        
        # Compute the following four normalized matrices.
        # - v_dot_d_find_pcs: genomic bin --> which PCs? 
        #    genomic bin squared contribution score.
        # - u_dot_d_fine_pcs: assay       --> which PCs? 
        #    assay squared contribution score.
        # - v_dot_d_find_loci: PC --> which genomic bins? 
        #    genomic bin contribution score.
        # - u_dot_d_find_assays: PC --> which assays? 
        #    assay contribution score        
        self.v_dot_d_find_pcs    = (v_dot_d ** 2 ) / (np.sum(v_dot_d ** 2, axis = 1)[:,np.newaxis])
        self.u_dot_d_find_pcs    = (u_dot_d ** 2 ) / (np.sum(u_dot_d ** 2, axis = 1)[:,np.newaxis])
        self.v_dot_d_find_loci   = (v_dot_d ** 2 ) / (np.sum(v_dot_d ** 2, axis = 0)[np.newaxis, :])
        self.u_dot_d_find_assays = (u_dot_d ** 2 ) / (np.sum(u_dot_d ** 2, axis = 0)[np.newaxis, :])
            
    def __find_generic(self, find_matrix, query_idx, topk=5):
        '''
        Generic form of "find_X_given_Y" function.
        '''
        return_idxs = np.argsort(-find_matrix[query_idx, :])[:topk]
        scores = find_matrix[query_idx, return_idxs]
        return return_idxs, scores
    
    def find_pcs_given_locus(self, locus_idx, topk=5):
        '''
        Given a specified locus_idx, return the top k PCs (default 5) and
        the corresponding their relative importance (genomic bin squared contribution score).
        '''
        return self.__find_generic(self.v_dot_d_find_pcs, locus_idx, topk)
    
    def find_pcs_given_assay(self, assay_idx, topk=5):
        '''
        Given a specified assay_idx, return the top k PCs (default 5) and
        the corresponding their relative importance (assay squared contribution score).
        '''
        return self.__find_generic(self.u_dot_d_find_pcs, assay_idx, topk)

    def find_loci_given_pc(self, pc_idx, topk=5):
        '''
        Given a specified PC index, return the top k genomic loci (default 5) and
        the corresponding their relative importance (genomic bin contribution score).
        '''
        return self.__find_generic(np.transpose(self.v_dot_d_find_loci), pc_idx, topk)
        
    def find_assays_given_pc(self, pc_idx, topk=5):
        '''
        Given a specified PC index, return the top k assays (default 5) and
        the corresponding their relative importance (assay contribution score).
        '''
        return self.__find_generic(np.transpose(self.u_dot_d_find_assays), pc_idx, topk)

    # multipe SNPs/assays mode
    def __find_given_list_generic(self, find_matrix, query_idxs, weights = None, topk=5):
        '''
        Generic form of "find_pcs_given_X_list" function.
        '''
        find_matrix_slice = find_matrix[query_idxs, :]
        if(weights is None):
            find_vec = np.sum(find_matrix_slice, axis = 0) / len(query_idxs)
        else:
            weights_vec = np.array(weights)[:, np.newaxis] / np.sum(weights)
            find_vec = np.sum(weights_vec * find_matrix_slice, axis = 0)            
        return_idxs = np.argsort(-find_vec)[:topk]
        scores = find_vec[return_idxs]
        return return_idxs, scores
    
    def find_pcs_given_loci_list(self, loci_idxs, weights = None, topk=5):
        '''
        Given indices of genomic loci as a list, return the top k PCs (default 5) and
        the corresponding their relative importance.
        One can also give weights for the list of loci.        
        '''
        return self.__find_given_list_generic(self.v_dot_d_find_pcs, loci_idxs, weights, topk)
        
    def find_pcs_given_assay_list(self, assay_idxs, weights = None, topk=5):
        '''
        Given indices of assays as a list, return the top k PCs (default 5) and
        the corresponding their relative importance.
        One can also give weights for the list of assays.
        '''
        return self.__find_given_list_generic(self.u_dot_d_find_pcs, assay_idxs, weights, topk)
    

In [76]:
data_dir = '/afs/cs.stanford.edu/u/ytanigaw/repos/lkalesinskas/bigtranscription/private_data'


In [83]:
decomposed_mats = Decomposed_matrices(
    os.path.join(data_dir, 'diagonalScore.csv.gz'),
    os.path.join(data_dir, 'uScore.csv.gz'),
    os.path.join(data_dir, 'vScore.csv.gz')
)

In [87]:
for locus in [40973, 114953, 275291, 216060]:
    print(locus, decomposed_mats.find_pcs_given_locus(locus))

40973 (array([ 0,  5,  2,  8, 39]), array([0.17388874, 0.14292424, 0.07116638, 0.05649573, 0.02586899]))
114953 (array([16,  7, 14, 59, 38]), array([0.06951082, 0.06718931, 0.04944998, 0.04625181, 0.03272474]))
275291 (array([5, 2, 8, 7, 3]), array([0.27117038, 0.09215098, 0.07264342, 0.06559204, 0.04081626]))
216060 (array([ 5,  1, 73, 14,  2]), array([0.10332161, 0.04327748, 0.03535886, 0.02969716, 0.02020912]))


#### query for multiple SNPS

In [106]:
pc_0index, pc_weights = decomposed_mats.find_pcs_given_loci_list(
    genomic_bins.find_loci_given_snps(
        [2, 5, 13, 10],
        [33701000, 1315000, 42952000, 6390000]
    ), 
    weights=[1, 1, 1, 1]
)

for i, dat in enumerate(zip(pc_0index, pc_weights)):
    print('Rank{}: PC{} (weight {:.4f})'.format(i, dat[0] + 1, dat[1]))

Rank0: PC6 (weight 0.1321)
Rank1: PC1 (weight 0.0596)
Rank2: PC3 (weight 0.0466)
Rank3: PC9 (weight 0.0371)
Rank4: PC8 (weight 0.0362)


#### query for single SNP

In [107]:
pc_0index, pc_weights = decomposed_mats.find_pcs_given_locus(
    genomic_bins.find_locus_given_snp(2, 33701000)
)

for i, dat in enumerate(zip(pc_0index, pc_weights)):
    print('Rank{}: PC{} (weight {:.4f})'.format(i, dat[0] + 1, dat[1]))

Rank0: PC1 (weight 0.1739)
Rank1: PC6 (weight 0.1429)
Rank2: PC3 (weight 0.0712)
Rank3: PC9 (weight 0.0565)
Rank4: PC40 (weight 0.0259)


In [84]:
decomposed_mats.find_pcs_given_loci_list(
    [40973, 114953, 275291, 216060]
)

(array([5, 0, 2, 8, 7]),
 array([0.13214054, 0.05960559, 0.04656923, 0.03709355, 0.036163  ]))

In [88]:
decomposed_mats.find_pcs_given_loci_list(
    [40973, 114953, 275291, 216060],
    weights=[1, 0, 0, 0]
)

(array([ 0,  5,  2,  8, 39]),
 array([0.17388874, 0.14292424, 0.07116638, 0.05649573, 0.02586899]))

In [89]:
decomposed_mats.find_pcs_given_loci_list(
    [40973, 114953, 275291, 216060],
    weights=[0, 1, 0, 0]
)

(array([16,  7, 14, 59, 38]),
 array([0.06951082, 0.06718931, 0.04944998, 0.04625181, 0.03272474]))

In [90]:
decomposed_mats.find_pcs_given_loci_list(
    [40973, 114953, 275291, 216060],
    weights=[0, 0, 1, 0]
)

(array([5, 2, 8, 7, 3]),
 array([0.27117038, 0.09215098, 0.07264342, 0.06559204, 0.04081626]))

In [91]:
decomposed_mats.find_pcs_given_loci_list(
    [40973, 114953, 275291, 216060],
    weights=[0, 0, 0, 1]
)

(array([ 5,  1, 73, 14,  2]),
 array([0.10332161, 0.04327748, 0.03535886, 0.02969716, 0.02020912]))

In [78]:
decomposed_mats.find_pcs_given_locus(328517)

(array([ 7, 39,  5, 41,  1]),
 array([0.07565113, 0.06052896, 0.05604721, 0.04363581, 0.03172747]))

In [79]:
decomposed_mats.find_assays_given_pc(7)

(array([188, 441, 433, 434, 439]),
 array([0.09254222, 0.08990549, 0.08470579, 0.06732091, 0.04859752]))

In [80]:
decomposed_mats.find_loci_given_pc(7)

(array([249953, 224313, 330510, 273660, 294763]),
 array([6.58942556e-05, 6.22829820e-05, 6.12494759e-05, 6.06528861e-05,
        5.92396118e-05]))

In [25]:
df=pd.DataFrame({'a': np.arange(10) * 2})

In [28]:
dict(zip(df['a'], df.index))

{0: 0, 2: 1, 4: 2, 6: 3, 8: 4, 10: 5, 12: 6, 14: 7, 16: 8, 18: 9}

In [55]:
class Genomic_bins:
    def read_genomic_bin_bed_file(self, genomic_bin_bed_file):
        '''
        Read bed file that defines genomic bins
        '''
        genomic_bin_df=pd.read_csv(
            genomic_bin_bed_file,
            names=['chr', 'chromStart', 'chromEnd', 'name'],
            sep='\t'
        )
        return genomic_bin_df
    
    def __init__(self, genomic_bin_bed_file, bin_size = 1000):
        '''
        Read the definition of the genomic loci BED file and load to memory
        '''
        self.genomic_bin = self.read_genomic_bin_bed_file(genomic_bin_bed_file)
        self.name_to_loci = dict(zip(self.genomic_bin['name'], self.genomic_bin.index))
        self.bin_size = bin_size
    
    def find_locus_given_snp(self, chrom, position):
        '''
        Given a genomic coordinate a pair of (chrom, position),
        return the index of the corresponding genomic locus.
        '''
        name='chr{}_{}'.format(chrom, int(position / self.bin_size))
        return self.name_to_loci[name]

    def find_loci_given_snps(self, chroms, positions):
        '''
        Given a list of genomic coordinates 
        (one need to pass as two lists of chromosome and positions),
        return the list of genomic loci
        '''
        return [
            self.find_locus_given_snp(x[0], x[1]) for x in
            zip(chroms, positions)
        ]

In [52]:
genomic_bins = Genomic_bins(
    os.path.join(data_dir, 'loci_def.bed')
)

In [37]:
genomic_bins.find_loci_given_snp(17, 38023745)

328517

In [56]:
genomic_bins.find_loci_given_snps(
    [2, 5, 13, 10],
    [33701000, 1315000, 42952000, 6390000]
)

[40973, 114953, 275291, 216060]

In [86]:
locus = genomic_bins.find_locus_given_snp(2, 33701000)
locus, decomposed_mats.find_pcs_given_locus(locus)

(40973,
 (array([ 0,  5,  2,  8, 39]),
  array([0.17388874, 0.14292424, 0.07116638, 0.05649573, 0.02586899])))

In [45]:
locus = genomic_bins.find_loci_given_snp(5, 1315000)
locus, decomposed_mats.find_pcs_given_locus(locus)

(114953,
 (array([16,  7, 14, 59, 38]),
  array([0.06951082, 0.06718931, 0.04944998, 0.04625181, 0.03272474])))

In [47]:
locus = genomic_bins.find_loci_given_snp(13, 42952000)
locus, decomposed_mats.find_pcs_given_locus(locus)

(275291,
 (array([5, 2, 8, 7, 3]),
  array([0.27117038, 0.09215098, 0.07264342, 0.06559204, 0.04081626])))

In [48]:
locus = genomic_bins.find_loci_given_snp(10, 6390000)
locus, decomposed_mats.find_pcs_given_locus(locus)

(216060,
 (array([ 5,  1, 73, 14,  2]),
  array([0.10332161, 0.04327748, 0.03535886, 0.02969716, 0.02020912])))